<a href="https://colab.research.google.com/github/vvithurshan/Homology-Modelling-for-Abcov-Dataset/blob/main/mutation_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
!pip install mdtraj
!pip install MDAnalysis

In [ ]:
df = pd.read_csv("./FYP_summarized_Dataset - AB-Bind.csv")

In [ ]:
# df.head()

In [ ]:
df = df[['PDB_Complex', 'Partners(A_B)', 'Mutation']]

In [ ]:
df.head()

In [ ]:
import random

random.seed(5000)

df['Unique_id'] = df['PDB_Complex'].apply(lambda x : x + '_' + str(random.randint(0, 100000)))

## Main Program Begins

In [ ]:
# !cd /content
!git clone https://github.com/sarahsirin/AB-Bind-Database.git

In [ ]:
import os
import subprocess
import MDAnalysis as mda
import mdtraj as md
import shutil

os.chdir("/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/march/")

# !rm -rf P001

project_name = "P001"
os.makedirs(project_name)
os.chdir(project_name)

for pdb in Unique_PDB[:]:

  # if '_' in pdb:
  #   pdb = pdb.split('_')[1]
## create a folder with name pdb - done
  os.makedirs(pdb)
  os.chdir(pdb)

## download pdb - done
  def DownloadPDB(code):
    # link = "wget https://files.rcsb.org/download/" + code + ".pdb"
    os.makedirs('rcsb')
    os.chdir('./rcsb')
    
    source_file = "/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/march/AB-Bind-Database/" + code + ".pdb"
    destination_folder = os.getcwd()
    shutil.copy(source_file, destination_folder)
    # subprocess.check_output(link, shell=True)
    os.chdir('../')  
  DownloadPDB(pdb)

## remove non protein atoms 

  def RemoveNonProteinAtoms(chains_):
    os.makedirs('template')
    os.chdir('./template')

    input = "../rcsb/" + pdb + ".pdb"
    universe = mda.Universe(input)
    protein_atoms = universe.select_atoms("protein") 

    chainsToExt = ' '.join(chains_)

## extract only relevant chains
    protein_atoms_required_chains = protein_atoms.select_atoms("segid " + chainsToExt)
    output = pdb + "_template.pdb"
## save as a template
    protein_atoms_required_chains.write(output)

    os.chdir('../')  

## store the sequence of the template, in dict form
  def TemplateToFasta(chains_, pdb):
    u = mda.Universe("./template/" + pdb + "_template.pdb")
    chainsToFasta = {}
    for chain in chains_:
      selection = 'protein and segid ' + chain
      chain_selection = u.select_atoms(selection)
      sequence = chain_selection.residues.sequence()
      chainsToFasta[chain] = str(sequence.seq)
    # return pd.Series(chainsToFasta)
    return chainsToFasta


#########
  chainsToExtract = df[df['PDB_Complex'] == pdb]['Partners(A_B)']
  chains = list(list(chainsToExtract)[0].replace('_',''))

  RemoveNonProteinAtoms(chains)
##########

  for index, data in chainsToExtract.iteritems():
    # print(index, data)
    df.loc[index, 'Template_Fasta']  = [TemplateToFasta(chains, pdb)]


    lst = df[df['PDB_Complex'] == pdb]['Mutation'][index].split(',')
    unique_id = df[df['PDB_Complex'] == pdb]['Unique_id'][index]

    CreateMutatedPDB(pdb, lst, unique_id)
    
    df.loc[index,'Mutated_Fasta'] = [MutatedToFasta(pdb, unique_id, chains)]

    # being written
    # HomologyModeling()

  os.chdir('../')  

# !cd /content
  

In [ ]:
pip install mdtraj

In [ ]:
### It creates mutated pdb files_hm_false
import warnings
from Bio.PDB import PDBParser, PPBuilder, Polypeptide, PDBIO

def CreateMutatedPDB(pdb, lst, unique_id):
  folder = pdb + "_mutation"
  if not os.path.exists(folder):
    os.mkdir(folder)
  os.chdir(folder)

  # Load template PDB file 
  parser = PDBParser()
  structure = parser.get_structure(pdb, "../template/{}_template.pdb".format(pdb))

  deletion = 0
  e = ""
  add = False
  for i in lst:
    print(i)
    ## it handles deletion operation
    if 'delta' in i:
      e += i
      add = True
    elif add:
      e += "," + i
      print(e)
      if add and ')' in i:
        add = False
        #call deletion
        a = e[e.find('('):].replace('(','').replace(')','').split(',')
        print(a)
        for i in a:
          chain_id = i.split(':')[0]
          resid_id = int(i.split(':')[1][1:])
          print("chain_id {} resid_id {}".format(chain_id, resid_id))
          chain_a = structure[0][chain_id]
          try:
            del chain_a[(" ", resid_id, " ")]
          except KeyError:
             print("resid {} is not removed".format(resid_id))
          else:
            deletion += 1

    else:

      I = i.split(':')
      mut_from = I[1][0]
      chain = I[0]
      try:
        resid = int(I[1][1:-1]) 
      except ValueError:
        resid = int(I[1][1:-2])
        mutation_res = I[1][-1]
        print("It has unwanted str")
      else:
        mutation_res = I[1][-1]

      # print(mut_from, chain, resid, mutation_res)
      # Find residue to mutate
      chain = structure[0][chain]
      try:   
        residue_id = (" ", resid, " ")  
        residue = chain[residue_id]
      except KeyError:
          print("Residue {} not found in the structure {}".format(resid, pdb))
          continue

      # Mutate residue
      warnings.filterwarnings("ignore")
      mutated_residue = mutation_res
      residue.resname = Polypeptide.one_to_three(mutated_residue) 
      warnings.filterwarnings("default")
  # Write new PDB file
  io = PDBIO()
  io.set_structure(structure)
  output = "{}_hm_false.pdb".format(unique_id)
  io.save(output)

  os.chdir("../")

In [ ]:
## it return the Fasta sequence of mutated pdb_hm_false

def MutatedToFasta(pdb ,unique_id, chains_):

  u = mda.Universe("./" + pdb + "_mutation/" + unique_id + "_hm_false.pdb")
  chainsToFasta = {}
  for chain in chains_:
    selection = 'protein and segid ' + chain
    chain_selection = u.select_atoms(selection)
    sequence = chain_selection.residues.sequence()
    chainsToFasta[chain] = str(sequence.seq)
  return chainsToFasta
